# [Module 4.3] API GATEWAY 사용하여 SageMaker Endpoint 호출

![Fig.4.3.APIGATEWAY_flow](img/Fig.4.3.APIGATEWAY_flow.png)
아래 블로그를 보면 위 그림에 해당되는 플로우를 구현할 수 있습니다. <br>

여기서는 아래와 같은 단계를 진행 합니다.
- SageMaker Endpoint 블러오기 (%store -r)
- 노트북에서 람다 함수 생성 및 테스트 
- 람다 콘솔에서 람다 함수 생성
- API GATEWAY에서 API GATEWAY 생성하기
- API GATEWAY에서 API GATEWAY 테스트 하기
- 외부 웹 페이지에서 테스트

---
Reference:
- [Blog] Call an Amazon SageMaker model endpoint using Amazon API Gateway and AWS Lambda
    - https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/


## Lambda 함수 만들기

Lambda 함수의 Signature(함수 정의)를 같게 하고, 필요한 것은 Dummy 값을 할당하여 구현 함.

In [8]:
%store -r

In [15]:
print("tweet_bert_endpoint_name: \n", tweet_bert_endpoint_name)
os.environ['ENDPOINT_NAME'] = tweet_bert_endpoint_name

tweet_bert_endpoint_name: 
 sagemaker-tensorflow-serving-2020-08-16-02-10-36-839


In [29]:
import json
import os
import io
import boto3
import csv

ENDPOINT_NAME = os.environ['ENDPOINT_NAME'] 
runtime = boto3.client('runtime.sagemaker')


def lambda_handler(event, context):
    # TODO implement
    try:
        print("Received event: " + json.dumps(event, indent=2))

        data = json.loads(json.dumps(event))
        payload = data['body']
        payload = '["' + payload + '"]'
        print("Processed inference input: \n" ,payload)

        response = runtime.invoke_endpoint(EndpointName = ENDPOINT_NAME,
                                        ContentType = 'application/json',
                                        Body = payload)

        result = json.loads(response['Body'].read().decode())
        print("\nlambda_response: \n ", result)        

        return {
            'statusCode': 200,
            'body': json.dumps(result)
        }        
    except:
        print("An error occurred")
        return data


## Lambda 함수 결과 보기
아래와 같이  람다 함수가 정상적으로 결과를 제공하면 람다 콘솔에 가서 람다를 만들면 됨
lambda_response 는 10개의 이모티콘의 Score를 보여주고 있고, Top 3 추천을 위해서는 
정렬하여 가장 높은 값 3개를 추천하면 됨
```
Received event: {
  "body": "I am good"
}
Processed inference input: 
 ["I am good"]

lambda_response: 
  [[1.07107985, 2.42111945, -2.42198849, -1.93775201, -1.99158204, -2.29052329, 4.34999323, -0.153068662, -1.22072053, -1.50339854]]
```

In [30]:
event = {'body' : 'I am good'}
context= "Dummy Context"
lambda_response = lambda_handler(event, context)   



Received event: {
  "body": "I am good"
}
Processed inference input: 
 ["I am good"]

lambda_response: 
  [[1.07107985, 2.42111945, -2.42198849, -1.93775201, -1.99158204, -2.29052329, 4.34999323, -0.153068662, -1.22072053, -1.50339854]]


## Lambda 콘솔에서 람다 함수 만들기

![Fig.4.3.lambda-function](img/Fig.4.3.lambda-function.png)

## Lambda 함수 테스트 및 결과 확인

![Fig.4.3Test-Lambda-Function](img/Fig.4.3Test-Lambda-Function.png)

![Fig.4.3.Result-Lambda-Test](img/Fig.4.3.Result-Lambda-Test.png)

## API GATEWAY 함수 생성
- 자세한 설명은 노트북 맨 위의 블로그를 참조 하세요.

여기서는 InvokeTweetEndpoint 람다 함수를 Integration Type으로 선택하고, 람다 함수 이름을 기입하여 생성 하였습니다.

![Fig.4.3.APIGATEWAY-Setting](img/Fig.4.3.APIGATEWAY-Setting.png)

## API GATEWAY 함수 내부 테스트

아래와 같이 입력값을 넣고 테스트를 하여 결과가 나와야 합니다.

![Fig.4.3.APIGATEWAY_test](img/Fig.4.3.APIGATEWAY_test.png)

## API GATEWAY 외부에서 테스트

아래와 같이 먼저 Invoke URL을 카피 합니다.

![Fig.4.3.APIGATEWAY-URL](img/Fig.4.3.APIGATEWAY-URL.png)

저는 아래 URL 에서 테스트를 수행하였습니다. 원하시는 Web API 테스트 웹사이트 혹은 애플리케이션을 이용하시면 됩니다.
API TESTERBE ( https://apitester.com )

## 외부 웹사이트 (https://apitester.com) 에서 API GATEWAY 테스트

위에서 복사한 Invoke URL을 카피하고, Post Data에 {"body": "I am Good"} 를 입력 합니다.
이후 테스트 버튼을 누르면 결과를 확인할 수 있습니다.

혹시 [AWS API Gateway: Solving Missing Authentication Tokens](http://www.awslessons.com/2017/aws-api-gateway-missing-authentication-token/) 에러가 빌생하면 클릭하여 정보 참고 하세요

![Fig.4.3.Result-External-Call](img/Fig.4.3.Result-External-Call.png)